### In process() ==> added avoid words
### Eliminate category: 'painting'
### Division by zero error -> solved

In [5]:
import pandas as pd
from IPython.display import display
import numpy as np
from collections import Counter
import nltk
import psycopg2
from nltk.corpus import stopwords
import re
from itertools import chain 
from IPython.display import Image
import math
import time
import logging 
import requests

In [2]:
from gensim import models
w = models.KeyedVectors.load_word2vec_format(r"F:\Pretrained Models\GoogleNews-vectors-negative300.bin.gz", binary=True)

In [3]:
def process(array,avoidwords):
    text = re.sub(r'\[[0-9]*\]',' ',str(array))  #Remove Numbers
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Remove nums
    text = re.sub(r'\s+',' ',text)  #Remove extra space
    text = re.sub(r"[^a-zA-Z0-9]+",' ',text)  #Remove special characters
    text = text.lower()  #Lower case all
    text = nltk.sent_tokenize(text)  #Tokenize to sentences 
    keywords = [nltk.word_tokenize(sentence) for sentence in text]
    stop_words = stopwords.words('english')
    stop_words.extend(avoidwords)
    for i in range(len(keywords)):
        keywords[i] = [word for word in keywords[i] if word not in stop_words]
    return keywords

#Frame pre-processing function
def prepro(frame,cap_or_bio,avoidwords):
    frame2 = frame
    for c in range(len(frame2)):
        keywords = process(frame2[cap_or_bio][c],avoidwords)
        frame2[cap_or_bio][c] = keywords
    return frame2

def flat(array):     # 2D array reduction
    flatten_list = [j for sub in array for j in sub] 
    flatten_list = list(chain.from_iterable(flatten_list))
    return flatten_list


# normalize() -> given an array, converts to 1/0, top int(pos) will be 1
def normalize(keys, pos =3):  
    ax = [i for i in keys]
    temp = [i for i in keys]
    temp.sort()
    temp = temp[-pos:]
    for x in temp:
        ax[keys.index(x)] = 1
    for x in range(len(ax)):
        if ax[x] != 1:
            ax[x] = 0
    return ax

def normalizeSD(keys, thre =3):    # Given score array return shortlisted cats in given threshold
    ax = deviation(keys)
    ax = dev_shortlist(ax)
    return ax

# compute() => category[] to be called outside
def compute(caption,category,top =3):
    ar = []
    score = []

    # Code to get frequency distribution and unique keywords array
    keywords = []
    caption_freq = []
    counts = Counter(caption)
    if len(counts) > 0:
        labels, values = zip(*counts.items())
        ## sort your values in descending order
        indSort = np.argsort(values)[::-1]
        ## rearrange your data
        keywords = np.array(labels)[indSort]  # Label
        caption_freq = np.array(values)[indSort]  # Values
    
    # Detect words not in Google Dict | Put freq = 0
    for x in keywords:
        try:
            restConst = w.similarity(x,'something')
        except KeyError:
            caption_freq[np.where(keywords == x)] = 0
        
    #Google similaity function
    for x in category:
        empty = []
        for y in keywords:
            try:
                empty.append(w.similarity(x,y))
            except:
                empty.append(0)
        ar.append(empty)
    
    # Store the similarity values in dataframe
    frame = pd.DataFrame()
    frame = pd.DataFrame(ar, columns = keywords)
    
    
    ### CHANGES  MADE
    #Normalize | top select
    for key in frame.columns:
        frame[key] = normalizeSD(frame[key].tolist(),top)
    
    # Multiply with frequency
    for row in range(len(frame)):
        frame.values[row] = [i*j for i,j in zip(frame.values[row],caption_freq)]
    # Sum the values => Score
    for row in range(len(frame)):
        score.append(sum(frame.values[row]))
    
    frame['category'] = category
    frame['Scores'] = score
    return frame,keywords[:6]

def deviation(array):
    mu = max(array)
    l = len(array)
    ar = []
    for x in range(l):
        ar.append(math.sqrt((array[x]-mu)**2)/l)
    total = sum(ar)
    for x in range(l):
        if total != 0:
            ar[x] = (ar[x]/total)*100
    return ar

def mean_deviation(array):
    l = len(array)
    mu = sum(array)/l
    ar = []
    for x in range(l):
        ar.append(math.sqrt((array[x]-mu)**2)/l)
    total = sum(ar)
    for x in range(l):
        if total != 0:
            ar[x] = (ar[x]/total)*100
    return ar

def dev_shortlist(dev_array,thre = 2):  # Shortlist using threshold from deviation array | return array in 1/0
    final_cat = [0]*len(dev_array)
    for i in range(len(dev_array)):
        if dev_array[i] <=thre:
            final_cat[i] = 1
    return final_cat

def sort_cat(dev,cat, thre = 2):   # Shortlist using thre | Return category array
    final_cat = []
    for i in range(len(dev)):
        if dev[i] <=thre:
            final_cat.append(cat[i])
    return final_cat

def get_row_pscore(col_name,f1,f2, scoreType):  # f1-mainframe | f2-frame
    ud = f1.at[0,'user_id']
    hd = f1.at[0,'handle']
    fl = f1.at[0,'followers']
    ul = f1.at[0,'url']
    row_in_array = [ud,hd,fl,ul]
    dev_array = f2[scoreType].tolist()
    row_in_array.extend(dev_array)
    tk = f2.at[0,'Top keywords']
    row_in_array.extend([tk])
    zip_it = zip(col_name,row_in_array)
    convert_to_dict = dict(zip_it)
    return convert_to_dict

def top_category(f2,categories,thre=2):
    final_cat = [0]*len(categories)
    dev_scores = f2['Deviation'].tolist()
    rank_of_cat = 1
    for i in range(len(dev_scores)):
        if dev_scores[i] <=thre:
            final_cat[i] = rank_of_cat 
            rank_of_cat = rank_of_cat +1
    return final_cat

def top_category_get_percent(f2,categories,thre=2):
    final_cat = [0]*len(categories)
    dev_scores = f2['Deviation'].tolist()
    percent_array = f2['Percentage'].tolist()
    for i in range(len(dev_scores)):
        if dev_scores[i] <=thre:
            final_cat[i] = percent_array[i]
    return final_cat
    
    
def get_row_result(col_name,f1,f2,rank_array):  # f1-mainframe
    ud = f1.at[0,'user_id']
    hd = f1.at[0,'handle']
    fl = f1.at[0,'followers']
    ul = f1.at[0,'url']
    row_in_array = [ud,hd,fl,ul]
    row_in_array.extend(rank_array)
    tk = f2.at[0,'Top keywords']
    row_in_array.extend([tk])
    zip_it = zip(col_name,row_in_array)
    convert_to_dict = dict(zip_it)
    return convert_to_dict

In [6]:
conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
cur = conn.cursor()

#### IDs from database

In [7]:
cur.execute('''SELECT  "instagram_accounts"."user_id", "instagram_accounts"."handle", "instagram_accounts"."followers", "instagram_accounts"."url" FROM "instagram_accounts" WHERE (followers>5000 and followers<30000 and country = 'India' and active = true) ORDER BY followers Desc''')
InstagramAccount = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])
InstagramAccount = InstagramAccount[5000:6200]
# InstagramAccount = pd.DataFrame(cur.fetchall(),columns=["user_id"])
# InstagramAccount = InstagramAccount[:10000]
Insta_ids = InstagramAccount['user_id'].tolist()
print("Total accounts taken: ",len(Insta_ids))

Total accounts taken:  1200


#### IDs from CSV

In [58]:
InstagramAccount = pd.read_csv(r'E:\Winkl Mains\DATA\Class 9 records\TRAVEL\TRAVELprofile_percentages.csv')
InstagramAccount = InstagramAccount[:100]
Insta_ids = InstagramAccount['user_id'].tolist()
print("Total accounts taken: ",len(Insta_ids))

Total accounts taken:  100


In [10]:
# Categories
categories = ['food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel','photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting']


# DataFrame Creation

profile_deviations = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel', 'photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting','top keywords'])
profile_mean_deviations = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel', 'photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting','top keywords'])
profile_percentages =  pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel', 'photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting','top keywords'])
onefive =  pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel', 'photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting','top keywords'])
twofive =  pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel', 'photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting','top keywords'])
threefive =  pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel', 'photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting','top keywords'])
twofive_in_percent =  pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel', 'photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting','top keywords'])
# This required to run some function

col_name = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'travel', 'photography','fitness','sports','gaming', 'entertainment', 'technology','investment','education', 'animal','wildlife', 'health', 'parenting','top keywords']

In [13]:
Insta_ids = [223704] # USE WHEN ==> For single account categorization

#Timer start
tic = time.perf_counter()
prev_tac = tic
txt = "Loaded {} accounts out of {}"

for x in Insta_ids:
    try:
        tac = time.perf_counter()
        time_compare = tac-prev_tac
        if time_compare >= 600:
            cur = conn.close()
            print("Connecting to Host...")
            import psycopg2
            conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
            cur = conn.cursor()
            prev_tac = tac
            print("Connected!")
            
        userid = x
        
        # Words which mostly occurs in insta post and we want to avoid considering them for the sake of accuracy of results
        avoidwords = ['verified','none']
        
        # Id, handle, url
        cur.execute('SELECT user_id, handle, url, followers  FROM instagram_accounts WHERE user_id = '+str(userid)+' ORDER BY user_id')
        mainframe = pd.DataFrame()
        mainframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])
        # captions
        cur.execute('SELECT "insta_user_posts"."insta_user_id", "insta_user_posts"."caption" FROM "insta_user_posts" WHERE (insta_user_id = '+str(userid)+')')
        randomframe = pd.DataFrame()
        randomframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "Caption"])

        if len(randomframe) ==0:
            raise Exception("No captions in Winkl database")
            
        r2 = pd.DataFrame()
        r2 = prepro(randomframe,'Caption',avoidwords)
        # Merge
        r3 = pd.DataFrame()
        r3 = r2.groupby('user_id').agg(lambda x: x.tolist())
        caption_array = r3['Caption'][x]
        # to 1D
        caption_array = flat(caption_array)
        #Temporary array i-> interim
        icaption_array = [i for i in caption_array]
        
        
        # Removing words not in dictionary
        for x in caption_array:
            try:
                checkword = w.similarity(x,'something')
            except KeyError:
                icaption_array.pop(icaption_array.index(x))
        #Restore Array
        caption_array = [i for i in icaption_array]
        
        if len(caption_array) ==0:
            raise Exception("No Words in profile for categorization or Different language")
            
        # Punishing accounts which has less than 1.5 words in caption
        if len(caption_array) < 2*len(randomframe):
            raise Exception("Too less words for categorization")
        
        # Word2vec computation
        frame = pd.DataFrame()
        frame, top_keywords = compute(caption_array,categories,3)
        
        #Convert to Percentage
        per = frame['Scores'].tolist()
        per_sum = sum(per)
        for x in range(len(per)):
            per[x] = round((per[x]/per_sum)*100)
        frame['Percentage'] = per
        # Deviation of categories
        dev = deviation(frame['Scores'].tolist())
        frame['Deviation'] = dev
        mu_dev = mean_deviation(frame['Scores'].tolist())
        frame['Mean Deviation'] = mu_dev
        frame['Top keywords'] = ', '.join(top_keywords)
        
        # Punishing accounts which has less than 1.5 words in caption
#         if len(caption_array) < 1.5*len(randomframe):
#             frame['Percentage'] = [0]*len(frame)
#             frame['Deviation'] = [0]*len(frame)
#             frame['Mean Deviation'] = [0]*len(frame)
#             frame['Scores'] = [0]*len(frame)
            
        # store profile deviation
        row_df_1 = get_row_pscore(col_name,mainframe,frame,'Deviation')
        profile_deviations = profile_deviations.append(row_df_1,ignore_index=True)
        
        # store profile deviation
        row_df_1_1 = get_row_pscore(col_name,mainframe,frame,'Mean Deviation')
        profile_mean_deviations = profile_mean_deviations.append(row_df_1_1,ignore_index=True)

        # store onefive result
        select_15_array = top_category(frame,categories,thre=1.5)
        row_df_2 = get_row_result(col_name,mainframe,frame,select_15_array)
        onefive = onefive.append(row_df_2,ignore_index=True)
        

        # store twofive result
        select_25_array = top_category(frame,categories,thre=2.5)
        row_df_3 = get_row_result(col_name,mainframe,frame,select_25_array)
        twofive = twofive.append(row_df_3,ignore_index=True)

        # store threefive result
        select_35_array = top_category(frame,categories,thre=3.5)
        row_df_4 = get_row_result(col_name,mainframe,frame,select_35_array)
        threefive = threefive.append(row_df_4,ignore_index=True)
        
        # Store profile percentage
        row_df_5 = get_row_pscore(col_name,mainframe,frame,'Percentage')
        profile_percentages = profile_percentages.append(row_df_5,ignore_index=True)
        
        # 2.5 categorize with percentage value
        select_25_array_percent = top_category_get_percent(frame,categories,thre=2.5)
        row_df_6 = get_row_result(col_name,mainframe,frame,select_25_array_percent)
        twofive_in_percent = twofive_in_percent.append(row_df_6,ignore_index=True)
        
        # Update on console
        print(txt.format(Insta_ids.index(userid),len(Insta_ids)))
        
    except Exception as Argument:
        
        # creating/opening a file 
        f = open(r"E:\Winkl Mains\DATA\Class 9 records\errorfile.txt", "a") 
        # writing in the file 
        f.write("In user_id:"+str(userid)+": "+str(Argument)+str("\n")) 
        # closing the file 
        f.close()  

print("profile_deviations")
display(profile_deviations)

print("profile_mean_deviations")
display(profile_mean_deviations)



print("profile_percentages")
display(profile_percentages)

print("onefive")
display(onefive)

print("twofive")
display(twofive)

print("threefive")
display(threefive)

print("twofive_in_percent")
display(twofive_in_percent)

# Timer end
toc = time.perf_counter()
print(f"Ran the model in {toc - tic:0.4f} seconds")

f = open(r"E:\Winkl Mains\DATA\Class 9 records\errorfile.txt", "a") 
# writing in the file 
f.write("The model ran in "+str(toc - tic)+" seconds"+str("\n")) 
# closing the file 
f.close() 

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Loaded 0 accounts out of 1
profile_deviations


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,gaming,entertainment,technology,investment,education,animal,wildlife,health,parenting,top keywords
0,223704,__e_s_h_a________,348385,https://www.instagram.com/__e_s_h_a________/,7.408885,6.850226,8.526204,0.75818,7.142857,2.527268,...,6.397978,0.0,8.526204,8.526204,6.810322,7.887736,7.116254,4.695398,5.413674,"kolkata, ig, calcutta, lanes, diaries, bengal"
1,223704,__e_s_h_a________,348385,https://www.instagram.com/__e_s_h_a________/,7.408885,6.850226,8.526204,0.75818,7.142857,2.527268,...,6.397978,0.0,8.526204,8.526204,6.810322,7.887736,7.116254,4.695398,5.413674,"kolkata, ig, calcutta, lanes, diaries, bengal"


profile_mean_deviations


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,gaming,entertainment,technology,investment,education,animal,wildlife,health,parenting,top keywords
0,223704,__e_s_h_a________,348385,https://www.instagram.com/__e_s_h_a________/,4.13251,3.056574,6.284381,8.676249,3.62016,5.269119,...,2.185579,10.136447,6.284381,6.284381,2.979722,5.054741,3.568925,1.093463,0.289882,"kolkata, ig, calcutta, lanes, diaries, bengal"
1,223704,__e_s_h_a________,348385,https://www.instagram.com/__e_s_h_a________/,4.13251,3.056574,6.284381,8.676249,3.62016,5.269119,...,2.185579,10.136447,6.284381,6.284381,2.979722,5.054741,3.568925,1.093463,0.289882,"kolkata, ig, calcutta, lanes, diaries, bengal"


...

threefive


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,gaming,entertainment,technology,investment,education,animal,wildlife,health,parenting,top keywords
0,223704,__e_s_h_a________,348385,https://www.instagram.com/__e_s_h_a________/,0,0,0,1,0,2,...,0,6,0,0,0,0,0,0,0,"kolkata, ig, calcutta, lanes, diaries, bengal"
1,223704,__e_s_h_a________,348385,https://www.instagram.com/__e_s_h_a________/,0,0,0,1,0,2,...,0,6,0,0,0,0,0,0,0,"kolkata, ig, calcutta, lanes, diaries, bengal"


twofive_in_percent


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,gaming,entertainment,technology,investment,education,animal,wildlife,health,parenting,top keywords
0,223704,__e_s_h_a________,348385,https://www.instagram.com/__e_s_h_a________/,0,0,0,13,0,0,...,0,14,0,0,0,0,0,0,0,"kolkata, ig, calcutta, lanes, diaries, bengal"
1,223704,__e_s_h_a________,348385,https://www.instagram.com/__e_s_h_a________/,0,0,0,13,0,0,...,0,14,0,0,0,0,0,0,0,"kolkata, ig, calcutta, lanes, diaries, bengal"


Ran the model in 2.2390 seconds


In [11]:
cat_name = 'Followlessthan30K-'  #keep it blank if dont want prefix
profile_deviations.to_csv(r'E:\Winkl Mains\DATA\Class 9 records\\'+cat_name+'profile_deviations.csv')
profile_percentages.to_csv(r'E:\Winkl Mains\DATA\Class 9 records\\'+cat_name+'profile_percentages.csv')
onefive.to_csv(r'E:\Winkl Mains\DATA\Class 9 records\\'+cat_name+'profile_cat_at_1.5.csv')
twofive.to_csv(r'E:\Winkl Mains\DATA\Class 9 records\\'+cat_name+'profile_cat_at_2.5.csv')
threefive.to_csv(r'E:\Winkl Mains\DATA\Class 9 records\\'+cat_name+'profile_cat_at_3.5.csv')
twofive_in_percent.to_csv(r'E:\Winkl Mains\DATA\Class 9 records\\'+cat_name+'profile_cat_in_percentage.csv')

In [66]:
# print(w.similarity("traveler","amazing"))
# print(w.similarity("beauty", "amazing"))
# # ars = [1,5,2,4,6,2,8,6,7,2,10]
# # ars.pop(ars.index(10))
# # print(ars)


-0.0046154372
0.28089756
